In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [2]:
df = pd.read_csv('train_data.csv')
df.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


In [3]:
df.drop(['case_id', 'patientid'], axis=1, inplace=True)

In [4]:
df.isna().sum().sum()

4645

In [5]:
df=df.dropna()

In [6]:
df.isna().sum().sum()

0

In [7]:
numeric_cols = df.select_dtypes(include=np.number)

In [8]:
correct_num = numeric_cols.drop(['Hospital_code', 'City_Code_Hospital', 'City_Code_Patient'], axis=1)

In [9]:
def outliersIQR(df, label):
    print('before=',df[label].shape)
    Q1 = np.percentile(df[label], 25,
                   interpolation = 'midpoint')
 
    Q3 = np.percentile(df[label], 75,
                   interpolation = 'midpoint')
    IQR = Q3 - Q1
    # Above Upper bound
    upper = Q3+1.5*IQR
    # Below Lower bound
    lower = Q1-1.5*IQR
    df_new = df[(df[label] < upper) & (df[label] > lower)]
    print('after=',df_new[label].shape)
    return df_new

In [10]:
for c in correct_num.columns:
    df=outliersIQR(df,c)
## this output bec >> more than run 


before= (313793,)
after= (309620,)
before= (309620,)
after= (309620,)
before= (309620,)
after= (297759,)
before= (297759,)
after= (286334,)


In [11]:
from sklearn.preprocessing import MinMaxScaler
def normMinMax(df,label):

    df[label] = MinMaxScaler().fit_transform(np.array(df[label]).reshape(-1,1))
    return df[label]

In [12]:
for c in numeric_cols.columns:
    print(df[c])
    df[c]=normMinMax(df,c)
    print(df[c])

0          8
1          2
2         10
4         26
5         23
          ..
318433     6
318434    24
318435     7
318436    11
318437    19
Name: Hospital_code, Length: 286334, dtype: int64
0         0.225806
1         0.032258
2         0.290323
4         0.806452
5         0.709677
            ...   
318433    0.161290
318434    0.741935
318435    0.193548
318436    0.322581
318437    0.580645
Name: Hospital_code, Length: 286334, dtype: float64
0         3
1         5
2         1
4         2
5         6
         ..
318433    6
318434    1
318435    4
318436    2
318437    7
Name: City_Code_Hospital, Length: 286334, dtype: int64
0         0.166667
1         0.333333
2         0.000000
4         0.083333
5         0.416667
            ...   
318433    0.416667
318434    0.000000
318435    0.250000
318436    0.083333
318437    0.500000
Name: City_Code_Hospital, Length: 286334, dtype: float64
0         3
1         2
2         2
4         2
5         2
         ..
318433    3
318434   

In [13]:
nominal_features = ['Hospital_type_code','Hospital_region_code','Department','Ward_Type','Ward_Facility_Code','Type of Admission']
ordinal_features = ['Severity of Illness','Age']

In [14]:
# One-Hot encoding of Nominal Features!

def one_hot_encode(dataset,features):
    nominal_df = dataset[features]

    # convert into dummies
    nominal_dummies = pd.get_dummies(nominal_df, drop_first=False)
    # drop categorical variables 
    dataset = dataset.drop(list(nominal_df.columns), axis=1)
    # concat dummy variables with X
    dataset = pd.concat([dataset, nominal_dummies], axis=1)
    return dataset

de_1_df = one_hot_encode(df,nominal_features)
de_1_df.head()


,Hospital_code,City_Code_Hospital,Available Extra Rooms in Hospital,Bed Grade,City_Code_Patient,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay,...,Ward_Type_U,Ward_Facility_Code_A,Ward_Facility_Code_B,Ward_Facility_Code_C,Ward_Facility_Code_D,Ward_Facility_Code_E,Ward_Facility_Code_F,Type of Admission_Emergency,Type of Admission_Trauma,Type of Admission_Urgent
0,0.225806,0.166667,0.500000,0.333333,0.162162,Extreme,0.333333,51-60,0.518341,0-10,...,0,0,0,0,0,0,1,1,0,0
1,0.032258,0.333333,0.333333,0.333333,0.162162,Extreme,0.333333,51-60,0.735723,41-50,...,0,0,0,0,0,0,1,0,1,0
2,0.290323,0.000000,0.333333,0.333333,0.162162,Extreme,0.333333,51-60,0.483743,31-40,...,0,0,0,0,0,1,0,0,1,0
4,0.806452,0.083333,0.333333,0.333333,0.162162,Extreme,0.333333,51-60,0.653189,41-50,...,0,0,0,0,1,0,0,0,1,0
5,0.709677,0.416667,0.333333,0.333333,0.162162,Extreme,0.333333,51-60,0.422051,11-20,...,0,0,0,0,0,0,1,0,1,0


In [15]:
from sklearn.preprocessing import OrdinalEncoder

ordinal = OrdinalEncoder()
ordinal_series_sev = ordinal.fit_transform(de_1_df[['Severity of Illness']])
ordinal_series_age = ordinal.fit_transform(de_1_df[['Age']])
de_2_df = de_1_df.drop(columns= ordinal_features)
de_2_df['Severity of Illness'] = ordinal_series_sev
de_2_df['Age'] = ordinal_series_age

In [16]:
de_2_df.head()

,Hospital_code,City_Code_Hospital,Available Extra Rooms in Hospital,Bed Grade,City_Code_Patient,Visitors with Patient,Admission_Deposit,Stay,Hospital_type_code_a,Hospital_type_code_b,...,Ward_Facility_Code_B,Ward_Facility_Code_C,Ward_Facility_Code_D,Ward_Facility_Code_E,Ward_Facility_Code_F,Type of Admission_Emergency,Type of Admission_Trauma,Type of Admission_Urgent,Severity of Illness,Age
0,0.225806,0.166667,0.500000,0.333333,0.162162,0.333333,0.518341,0-10,0,0,...,0,0,0,0,1,1,0,0,0.0,5.0
1,0.032258,0.333333,0.333333,0.333333,0.162162,0.333333,0.735723,41-50,0,0,...,0,0,0,0,1,0,1,0,0.0,5.0
2,0.290323,0.000000,0.333333,0.333333,0.162162,0.333333,0.483743,31-40,0,0,...,0,0,0,1,0,0,1,0,0.0,5.0
4,0.806452,0.083333,0.333333,0.333333,0.162162,0.333333,0.653189,41-50,0,1,...,0,0,1,0,0,0,1,0,0.0,5.0
5,0.709677,0.416667,0.333333,0.333333,0.162162,0.333333,0.422051,11-20,1,0,...,0,0,0,0,1,0,1,0,0.0,5.0


In [17]:
from sklearn.preprocessing import LabelEncoder
LabelEncode = LabelEncoder()
de_2_df['Stay'] = LabelEncode.fit_transform(de_2_df['Stay'])
de_2_df.Stay.value_counts(sort=True)

2     81371
1     73197
3     50091
5     30224
0     22216
4     10660
7      8046
8      3238
10     2922
6      2325
9      2044
Name: Stay, dtype: int64

In [18]:
X = de_2_df.drop(columns= ['Stay'])
y = de_2_df[['Stay']]


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.8,
                                                    test_size = 0.2, random_state=42)

In [20]:
from imblearn.over_sampling import SMOTE
print(y.shape[0])
oversample = SMOTE()
X_sampled, y_sampled = oversample.fit_resample(X_train, y_train)
print(y_sampled.value_counts())
print('increased= ',y_sampled.shape[0]-y.shape[0])

286334
Stay
0       65256
1       65256
2       65256
3       65256
4       65256
5       65256
6       65256
7       65256
8       65256
9       65256
10      65256
dtype: int64
increased=  431482


In [28]:
import time
## Grid search on all parameters took infinite time to run:) so by research and studing the docmentation we conclude that these are the best combinations
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
logModel = LogisticRegression(penalty='l2',solver='newton-cg',max_iter=10000,n_jobs=-1)
t1 = time.perf_counter()
logModel.fit(X_train, y_train.values.ravel())
t2 = time.perf_counter()
print('time', t2-t1)
####
print (f'Accuracy - : {logModel.score(X_test,y_test):.3f}')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


time 667.8699520449999
Accuracy - : 0.391


In [29]:
import time
## Grid search on all parameters took infinite time to run:) so by research and studing the docmentation we conclude that these are the best combinations
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
logModel = LogisticRegression(penalty='l2',solver='newton-cg',max_iter=10000,n_jobs=-1)
t1 = time.perf_counter()
logModel.fit(X_train, y_train.values.ravel())
t2 = time.perf_counter()
print('time', t2-t1)
####
print (f'Accuracy - : {logModel.score(X_test,y_test):.3f}')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


time 664.1538736570001
Accuracy - : 0.391


In [30]:
## Grid search on all parameters took infinite time to run:) so by research and studing the docmentation we conclude that these are the best combinations
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
logModel = LogisticRegression(penalty='l2',solver='lbfgs',max_iter=10000)
t1 = time.perf_counter()
logModel.fit(X_train, y_train.values.ravel())
t2 = time.perf_counter()
print('time', t2-t1)
####
print (f'Accuracy - : {logModel.score(X_test,y_test):.3f}')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


time 526.4537968179998
Accuracy - : 0.391


In [31]:
## Grid search on all parameters took infinite time to run:) so by research and studing the docmentation we conclude that these are the best combinations
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
logModel7 = LogisticRegression(penalty='none',solver='lbfgs',max_iter=10000,n_jobs=-1)
t1 = time.perf_counter()
logModel7.fit(X_train, y_train.values.ravel())
t2 = time.perf_counter()
print('time', t2-t1)
####
print (f'Accuracy - : {logModel7.score(X_test,y_test)}')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


time 301.5073332799998
Accuracy - : 0.3912375364520579


In [32]:
## Grid search on all parameters took infinite time to run:) so by research and studing the docmentation we conclude that these are the best combinations
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
logModel3 = LogisticRegression(penalty='l2',solver='liblinear',max_iter=10000,n_jobs=-1)
t1 = time.perf_counter()
logModel3.fit(X_train, y_train.values.ravel())
t2 = time.perf_counter()
print('time', t2-t1)
####
print (f'Accuracy - : {logModel3.score(X_test,y_test):.3f}')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1526: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs))


time 43.492888759999914
Accuracy - : 0.385


In [33]:
## Grid search on all parameters took infinite time to run:) so by research and studing the docmentation we conclude that these are the best combinations
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
logModel4 = LogisticRegression( penalty='l2',solver='sag',max_iter=10000,n_jobs=-1)
t1 = time.perf_counter()
logModel4.fit(X_train, y_train.values.ravel())
t2 = time.perf_counter()
print('time', t2-t1)
####
print (f'Accuracy - : {logModel4.score(X_test,y_test):.3f}')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


time 97.06853827699979
Accuracy - : 0.391


In [34]:
## Grid search on all parameters took infinite time to run:) so by research and studing the docmentation we conclude that these are the best combinations
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
logModel5 = LogisticRegression(penalty='none',solver='sag',max_iter=10000,n_jobs=-1)
t1 = time.perf_counter()
logModel5.fit(X_train, y_train.values.ravel())
t2 = time.perf_counter()
print('time', t2-t1)
####
print (f'Accuracy - : {logModel5.score(X_test,y_test):.3f}')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


time 435.1072275060001
Accuracy - : 0.391


In [35]:
## Grid search on all parameters took infinite time to run:) so by research and studing the docmentation we conclude that these are the best combinations
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
logModel6 = LogisticRegression(penalty='l2',solver='lbfgs',max_iter=10000,n_jobs=-1)
t1 = time.perf_counter()
logModel6.fit(X_train, y_train.values.ravel())
t2 = time.perf_counter()
print('time', t2-t1)
####
print (f'Accuracy - : {logModel6.score(X_test,y_test):.3f}')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


time 487.62248195300026
Accuracy - : 0.391


In [36]:
## Grid search on all parameters took infinite time to run:) so by research and studing the docmentation we conclude that these are the best combinations
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
logModel = LogisticRegression(penalty='none',solver='lbfgs',max_iter=10000,n_jobs=-1)
t1 = time.perf_counter()
logModel.fit(X_train, y_train.values.ravel())
t2 = time.perf_counter()
print('time', t2-t1)
####
print (f'Accuracy - : {logModel.score(X_test,y_test):.3f}')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


time 304.6963478589996
Accuracy - : 0.391


In [ ]:
import time
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
# SGDClf = linear_model.SGDClassifier(max_iter = 10000, tol=1e-3,penalty = "elasticnet")
t1 = time.perf_counter()
params = {
    "loss" : ["hinge", "log", "squared_hinge", "modified_huber"],
    "alpha" : [0.0001, 0.001, 0.01],
    "penalty" : ["l2", "l1", "none"],
}
model = SGDClassifier(max_iter=10000,n_jobs=-1)
clf = GridSearchCV(model, param_grid=params)
clf.fit(X_train, y_train.values.ravel())
print(clf.best_score_)
print(clf.best_estimator_)
print('time', t2-t1)
print (f'Accuracy - : {clf.score(X_test,y_test):.3f}')